This file has been cleaned up following finetuning and pretraining and is specific for a dataset combination that worked to give a positive R^2 value.

| Run Name                                           | R²    | Pearson | Description                                      | ATAC Data Source                                                                                  | RNA Data Source                                                                                   |
|----------------------------------------------------|-------|---------|--------------------------------------------------|--------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------|
| training_from_finetune_lora_chr10_chr11_split_QATAC_multiome_diff_RNA_no_treatment | 0.567 | 0.011   | Single-cell ATAC sequencing of MCF-7 treated with estrogen and progesterone | [GSE154873](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE154873) | [GSM6722730](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM6722730) |

| TSS Extension | TSS Used | RNA Normalisation      | ATAC Normalisation      | Leave-Out Chromosomes | Justification         | Notes                                      |
|---------------|----------|-----------------------|------------------------|----------------------|----------------------|--------------------------------------------|
| 300 bp        | Yes      | log10(TPM * 1e6 + 1) | log10(CPM * 1e5 + 1)  | chr10, chr11         | Recommendation by FuXi | First positive R² and a very high Spearman |


In [ ]:
#%%
from pathlib import Path

import numpy as np
import snapatac2 as snap
from gcell._settings import get_setting

annotation_dir = get_setting('annotation_dir')
print("gcell currently using annotation directory:", annotation_dir)

wget "http://catlas.org/catlas_downloads/humantissues/cCRE_hg38.tsv.gz" for tutorial cCRE file.

Followed by `gunzip` and renaming if desired.

In [ ]:
# Import necessary libraries
import pandas as pd
from pyranges import PyRanges as pr

# Define column names for the cCRE dataset
cols = ["Chromosome", "Start", "End", "Name", "Score", "Strand", "SignalValue", "PValue", "QValue", "SummitOffset", "Color", "DNase", "Classification"]

# Load cCRE data into a pandas DataFrame
cre = pd.read_csv('cCRE_hg38_tutorial.bed', sep='\t', header=None, names=cols)

# Keep only relevant columns for further processing
cre = cre[["Chromosome", "Start", "End"]]

# Convert the DataFrame to a PyRanges object with 64-bit integer precision and sort genomic intervals
cre = pr(cre, int64=True).sort()

# Display the first few rows of the sorted PyRanges object
cre.head()

### Candidate Cis-Regulatory Elements
Candidate cis-regulatory elements are genomic regions identified as having potential regulatory functions, such as enhancers or promoters, that control gene expression. The file being referenced (cCRE_hg38_tutorial.bed) contains information about these elements for the human genome (hg38 build).

__What Has Been Done so Far?__
1. Read in a cCRE file. (It's in a compressed from and separated by tabs) 
2. Renamed the columns and selected only the first three.
3. Converted the Pandas Dataframe to a Pranges object that is commonly used for genomic data and workflows (e.g., finding overlaps, merging regions). 
4. Then sorted the Pyranges object by Chromosome and then Start position.
5. Viewed first few lines to ensure all looks good.

ATAC_Counts file: https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE154873&format=file&file=GSE154873%5FMCF7%5Fmultiome%5FATAC%5Fcounts%2Etsv%2Egz

> Will not be using RNA counts from same source as following finetuning yields negative R^2 values.

### ATAC-Seq File

Peak accessibility file (.atac.bed) containing:  
Peak coordinates (chr, start, end)  
Normalized accessibility scores (aCPM, 10^5 scale factor) 

In [ ]:
import pandas as pd
import numpy as np

# Load ATAC counts
atac_counts = pd.read_csv("GSE154873_MCF7_multiome_ATAC_counts.tsv", sep="\t", index_col=0)

# Step 1: Extract Chromosome, Start, and End from the index
atac_counts.index = atac_counts.index.str.split("-", expand=True)
atac_counts.index.names = ["Chromosome", "Start", "End"]

# Convert Start and End to integers for proper numerical operations
atac_counts.index = atac_counts.index.set_levels([
    atac_counts.index.levels[0],  # Chromosome remains as string
    atac_counts.index.levels[1].astype(int),
    atac_counts.index.levels[2].astype(int)
])

# Step 2: Filter for chromosomes chr1–chr22 and chrX
valid_chromosomes = ["chr" + str(i) for i in range(1, 23)] + ["chrX"]
filtered_atac_counts = atac_counts[atac_counts.index.get_level_values("Chromosome").isin(valid_chromosomes)]

# Step 3: Calculate total counts for each peak across all cells
total_counts_per_peak = filtered_atac_counts.sum(axis=1)

# Step 4: Convert to aCPM using log10 transformation
acpm_values = np.log10(total_counts_per_peak / total_counts_per_peak.sum() * 1e5 + 1)

# Step 5: Create a DataFrame with Chromosome, Start, End, and aCPM
acpm_df = pd.DataFrame({
    "Chromosome": filtered_atac_counts.index.get_level_values("Chromosome"),
    "Start": filtered_atac_counts.index.get_level_values("Start"),
    "End": filtered_atac_counts.index.get_level_values("End"),
    "aCPM": acpm_values.values
})

# Step 6: Save the resulting aCPM DataFrame to a file
acpm_df.to_csv("atac_acpm_values.bed", sep="\t", index=False, header=None)

peaks = pr(acpm_df, int64=True)

print("Filtered ATAC aCPM values saved as 'atac_acpm_values.bed'.")

### FuXi Note:
For optimal zero-shot analysis, it is recommended to use a union set of peaks from both the new dataset and the original training peak sets *(i.e. the cre peaks defined above)* to minimize domain shift. However, in this tutorial we will simply perform finetuning using the new peak set from the 10x PBMC multiome data. The union peak set can be constructed as below. 

After that you should re-count the peak count matrix using fragment file, and when write out cell-type specific peaks, ideally remove all non-accessible peaks in each cell types.

In [ ]:
# Step 1: Find peaks that don't overlap with cCRE
non_overlap_peaks = peaks.overlap(cre, invert=True)

# Step 2: Concatenate non-overlapping peaks with cCRE to form a union set
total_peaks = pd.concat([non_overlap_peaks.df, cre.df], ignore_index=True)

# Convert the concatenated DataFrame back to PyRanges and sort it
total_peaks = pr(total_peaks, int64=True).sort()

# Step 3: Remove unwanted chromosomes (chrM, chrY, chrUn)
total_peaks = total_peaks.df.query('Chromosome.str.startswith("chr") & ~Chromosome.str.endswith("M") & ~Chromosome.str.endswith("Y") & ~Chromosome.str.startswith("chrUn")')

# Output the shape of the final peak set
print(f"Final peak set contains {total_peaks.shape[0]} peaks.")

# Step 4: Save the resulting union peak set to a file
total_peaks.to_csv("union_peaks_MCF7_with_cre.bed", sep="\t", index=False, header=False)
print("Union peak set saved as 'union_peaks_MCF7_with_cre.bed'.")

### What is the Purpose of This?

In this code, cre represents candidate cis-regulatory elements (genomic regions with potential regulatory roles), while peaks represents regions of open chromatin from the experimental data, inferred from ATAC-seq or other assays.

The overlapping is done to identify non-overlapping peaks using:

This step filters peaks that do not overlap with the cCRE regions. The purpose is to isolate peaks not previously annotated as regulatory elements, which may contain novel or unique regions worth further analysis.

So now this total_peaks (or union set) variable contains the peaks in MCF7 dataset that don't overlap with cre, and cre.

### Peak Length Visualisation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure with two subplots side by side (1 row, 2 columns)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot the length distribution of MCF7 peaks (peak length = End - Start) on the first subplot
sns.histplot(peaks.End - peaks.Start, ax=ax1)

#or if using union set
#sns.histplot(total_peaks.End - total_peaks.Start, ax=ax1)

ax1.set_title('MCF7 Peaks Length Distribution')

# Plot the length distribution of candidate cis-regulatory elements (CREs) on the second subplot
sns.histplot(cre.End - cre.Start, ax=ax2)
ax2.set_title('Training CRE Length Distribution')

# Adjust the layout to prevent overlapping elements
plt.tight_layout()

### Library Visualisation

In [ ]:
# Convert PyRanges object to a Pandas DataFrame
peaks_df = peaks.df  

# Count the number of peaks per chromosome
chromosome_peak_counts = peaks_df.groupby('Chromosome', observed=False).size().to_dict()

# Identify chromosomes with >100 peaks and total read count > 1M
chromosomes_for_modeling = []
print("Chromosomes with more than 100 peaks and total read count > 1M:")

# Iterate over each chromosome and check criteria
for chrom, peak_count in chromosome_peak_counts.items():
    if peak_count > 100:
        # Calculate the total read count (library size) for the chromosome
        libsize = peaks_df[peaks_df.Chromosome == chrom]["Counts"].sum()
        
        if libsize > 1_000_000:  # Library size threshold set to 1M for this dataset
            chromosomes_for_modeling.append(chrom)
            print(f"{chrom}: peaks = {peak_count}, library size = {libsize}")

## 2. Export Training Data

ATAC-Seq bed file previously prepared.

### FuXi Note:
Gene expression file (.rna.csv) containing:  
Gene names  
Normalized expression values (TPM, 10^6 scale factor)

In [ ]:
import pandas as pd
import numpy as np
from pyranges import PyRanges as pr

def process_atac_counts(input_file, output_file="atac_acpm_values.bed"):
    """
    Processes ATAC-seq count data to calculate average counts per million (aCPM)
    and saves the results as a BED file.
    
    Parameters:
        input_file (str): Path to the input ATAC count data file.
        output_file (str): Path to the output BED file (default: 'atac_acpm_values.bed').
        
    Returns:
        PyRanges: PyRanges object containing the processed aCPM values.
    """
    # Load ATAC counts
    atac_counts = pd.read_csv(input_file, sep="\t", index_col=0)

    # Extract Chromosome, Start, and End from the index
    atac_counts.index = atac_counts.index.str.split("-", expand=True)
    atac_counts.index.names = ["Chromosome", "Start", "End"]

    # Convert Start and End to integers for proper numerical operations
    atac_counts.index = atac_counts.index.set_levels([
        atac_counts.index.levels[0],  # Chromosome remains as string
        atac_counts.index.levels[1].astype(int),
        atac_counts.index.levels[2].astype(int)
    ])

    # Filter for chromosomes chr1–chr22 and chrX
    valid_chromosomes = ["chr" + str(i) for i in range(1, 23)] + ["chrX"]
    filtered_atac_counts = atac_counts[atac_counts.index.get_level_values("Chromosome").isin(valid_chromosomes)]

    # Calculate total counts for each peak across all cells
    total_counts_per_peak = filtered_atac_counts.sum(axis=1)

    # Convert to aCPM using log10 transformation
    acpm_values = np.log10(total_counts_per_peak / total_counts_per_peak.sum() * 1e5 + 1)

    # Create a DataFrame with Chromosome, Start, End, and aCPM
    acpm_df = pd.DataFrame({
        "Chromosome": filtered_atac_counts.index.get_level_values("Chromosome"),
        "Start": filtered_atac_counts.index.get_level_values("Start"),
        "End": filtered_atac_counts.index.get_level_values("End"),
        "aCPM": acpm_values.values
    })

    # Save the resulting aCPM DataFrame to a file
    acpm_df.to_csv(output_file, sep="\t", index=False, header=None)

    print(f"Filtered ATAC aCPM values saved as '{output_file}'.")

    # Return as PyRanges object for further use
    return pr(acpm_df, int64=True)

# Example usage
peaks = process_atac_counts("GSE154873_MCF7_multiome_ATAC_counts.tsv")

In [ ]:
import matplotlib.pyplot as plt

# Compute peak length by subtracting Start from End
peaks["length"] = peaks["End"] - peaks["Start"]

# Create a scatter plot for aCPM vs peak length
peaks.plot(
    x='aCPM', 
    y='length', 
    kind='scatter', 
    s=2,  # Marker size
    alpha=0.7,  # Transparency for better visualization
    title="Scatter Plot of aCPM vs Peak Length"
)

# Display the plot
plt.xlabel("aCPM")
plt.ylabel("Peak Length")
plt.show()

### Load RNA Data and Get Normalized TPM

In [ ]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv("GSE154873_MCF7_multiome_RNA_counts.tsv", sep="\t", index_col=0)

# Reset the index to move gene names into a column
df.reset_index(inplace=True)
df.rename(columns={"index": "gene_name"}, inplace=True)

# Remove periods from gene names and remove duplicates
df["gene_name"] = df["gene_name"].str.replace(".", "", regex=False)
df = df.drop_duplicates(subset="gene_name")

# Sum the total counts for each gene across all samples
total_counts_per_gene = df.iloc[:, 1:].sum(axis=1)

# Apply the provided TPM formula
df["TPM"] = np.log10((total_counts_per_gene / total_counts_per_gene.sum()) * 1e6 + 1)

# Keep only 'gene_name' and 'TPM' columns
final_df = df[["gene_name", "TPM"]]

# Save to 'rna_tpm_values.csv'
final_df.to_csv("rna_tpm_values.csv", index=False)

print("TPM calculation completed and saved as 'rna_tpm_values.csv'.")

## 3. Query motifs and save data as zarr files.

In [ ]:
# NOTE: tabix has to be >= 1.17
! tabix --version

In [ ]:
# Import the necessary libraries and modules
import os
from pathlib import Path

# Import specific functions from the 'gcell._settings' module and 'preprocess_utils' module
from gcell._settings import get_setting
from preprocess_utils import (
    add_atpm,        # Function to add ATPM (likely a type of data or calculation)
    add_exp,         # Function to add expression data
    create_peak_motif,  # Function to create peak motif data
    download_motif,     # Function to download motif data
    get_motif,          # Function to retrieve motif data
    query_motif,        # Function to query motif data
)

# Get the annotation directory setting using the 'get_setting' function and convert it to a Path object
annotation_dir = Path(get_setting('annotation_dir'))

# Print the directory being used for annotations (retrieved from the configuration)
print("gcell currently using annotation directory:", annotation_dir)

### Download motif bed file

In [ ]:
motif_bed_url = "https://resources.altius.org/~jvierstra/projects/motif-clustering/releases/v1.0/hg38.archetype_motifs.v1.0.bed.gz"
motif_bed_index_url = "https://resources.altius.org/~jvierstra/projects/motif-clustering/releases/v1.0/hg38.archetype_motifs.v1.0.bed.gz.tbi"


if (
    motif_bed_url
    and motif_bed_index_url
    and not (
        (annotation_dir / "hg38.archetype_motifs.v1.0.bed.gz").exists()
        or (annotation_dir / "hg38.archetype_motifs.v1.0.bed.gz.tbi").exists()
    )
):
    download_motif(motif_bed_url, motif_bed_index_url, motif_dir=annotation_dir)
    motif_bed = str(annotation_dir / "hg38.archetype_motifs.v1.0.bed.gz")
else:
    motif_bed = str(annotation_dir / "hg38.archetype_motifs.v1.0.bed.gz")

### Query Motif

In [ ]:
peak_bed = "atac_acpm_values.bed"  # Assuming peak files are named per chromosome

if os.path.exists(peak_bed):  # Ensure the peak file exists
        # Query motif file for the current chromosome's peak file
        peaks_motif = query_motif(peak_bed, motif_bed)

        # Process the queried motif data
        motif_output = get_motif(peak_bed, peaks_motif)

## 4. Create Peak Motif .Zarr File

Create a peak x motif matrix stored in a zarr file.

In [ ]:
create_peak_motif("get_motif.bed", "get_preprocess_output.zarr", peak_bed)

### Add aCPM data to region x motif matrix

In [ ]:
add_atpm(
        "get_preprocess_output.zarr",
        "atac_acpm_values.bed",
        "all_chrs",
    )

### Add Expression and TSS Data to Region x Motif Matrix

In [ ]:
add_exp(
        "get_preprocess_output.zarr", 
        "rna_tpm_values.csv", 
        "atac_acpm_values.bed", 
        "all_chrs", 
        assembly="hg38",
        version=44,
        extend_bp=300,  # Extend TSS region to 300bp upstream and downstream when overlapping with peaks
        id_or_name="gene_name",  # Use 'gene_name' from RNA data
    )

This has conclude the data processing steps.